In [13]:
import os
from multiprocessing import Pool

import nltk
from nltk.translate.bleu_score import SmoothingFunction
class SelfBleu():
    def __init__(self, test_text='', gram=3):
        super().__init__()
        self.name = 'Self-Bleu'
        self.test_data = test_text
        self.gram = gram
        self.sample_size = 500
        self.reference = None
        self.is_first = True

    def get_name(self):
        return self.name

    def get_score(self, is_fast=True, ignore=False):
        if ignore:
            return 0
        if self.is_first:
            self.get_reference()
            self.is_first = False
        if is_fast:
            return self.get_bleu_fast()
        return self.get_bleu_parallel()

    def get_reference(self):
        if self.reference is None:
            reference = list()
            with open(self.test_data) as real_data:
                for text in real_data:
                    text = nltk.word_tokenize(text)
                    reference.append(text)
            self.reference = reference
            return reference
        else:
            return self.reference

    def get_bleu(self):
        ngram = self.gram
        bleu = list()
        reference = self.get_reference()
        weight = tuple((1. / ngram for _ in range(ngram)))
        with open(self.test_data) as test_data:
            for hypothesis in test_data:
                hypothesis = nltk.word_tokenize(hypothesis)
                bleu.append(nltk.translate.bleu_score.sentence_bleu(reference, hypothesis, weight,
                                                                    smoothing_function=SmoothingFunction().method1))
        return sum(bleu) / len(bleu)

    def calc_bleu(self, reference, hypothesis, weight):
        return nltk.translate.bleu_score.sentence_bleu(reference, hypothesis, weight,
                                                       smoothing_function=SmoothingFunction().method1)

    def get_bleu_fast(self):
        reference = self.get_reference()
        # random.shuffle(reference)
        reference = reference[0:self.sample_size]
        return self.get_bleu_parallel(reference=reference)

    def get_bleu_parallel(self, reference=None):
        ngram = self.gram
        if reference is None:
            reference = self.get_reference()
        weight = tuple((1. / ngram for _ in range(ngram)))
        pool = Pool(os.cpu_count())
        result = list()
        sentence_num = len(reference)
        for index in range(sentence_num):
            hypothesis = reference[index]
            other = reference[:index] + reference[index+1:]
            result.append(pool.apply_async(self.calc_bleu, args=(other, hypothesis, weight)))

        score = 0.0
        cnt = 0
        for i in result:
            score += i.get()
            cnt += 1
        pool.close()
        pool.join()
        return score / cnt


In [14]:
import re
import os
model='vicuna'
folders=['_with_cosdecay_and_lasthiddenstate','_without_cosdecay_and_lasthiddenstate','_with_cosdecay_and_mean_sentemb','_without_cosdecay_and_mean_sentemb']
alpha=0
corpus_path = "vicuna_0_corpus.txt"
if os.path.exists(corpus_path):
    os.remove(corpus_path)

for folder in folders:
    rounds=os.listdir(model+folder)
    eval_paths=[ os.path.join(model+folder,round_,'eval_scripts','harmbench') for round_ in rounds]
    file_path=list()
    for eval_path in eval_paths:
        for file_name in os.listdir(eval_path):
            alpha_=int(re.findall(r'(?:search|alpha)_(\d*)_',file_name)[0])
            if alpha_==alpha:
                break
        file_path.append(os.path.join(eval_path,file_name))

    corpus=list()
    for file_name in file_path:
        corpus_single_file=list()
        judge_single_file=list()
        with open(file_name) as f:
            for line in f:
                if "show batch outputs" in line:
                    corpus_single_file.append(line[22:-4].strip().replace("\\'","'").replace("\\\\","\\").replace("\\\\","\\"))
                elif "completion" in line:
                    if line[13:-3] == "Yes":
                        judge_single_file.append(True)
                    else:
                        judge_single_file.append(False)
        jailbreak_item=[data for data, is_true in zip(corpus_single_file[-100:], judge_single_file[-100:]) if is_true]
        corpus.extend(jailbreak_item)

    with open(corpus_path, "a") as file:
        for item in corpus:
            file.write(item + "\n")

self_bleu = SelfBleu(test_text=corpus_path,gram=5)

# 计算 Self-BLEU 分数（完整计算）
score_full = self_bleu.get_score(is_fast=False)
print(f"Self-BLEU (Full): {score_full}")

Self-BLEU (Full): 0.6179531310914781


In [15]:
def get_self_bleu(model='vicuna',
folder='_with_cosdecay_and_lasthiddenstate',
alpha=10):
    rounds=os.listdir(model+folder)

    eval_paths=[ os.path.join(model+folder,round_,'eval_scripts','harmbench') for round_ in rounds]
    file_path=list()
    for eval_path in eval_paths:
        for file_name in os.listdir(eval_path):
            alpha_=int(re.findall(r'(?:search|alpha)_(\d*)_',file_name)[0])
            if alpha_==alpha:
                break
        file_path.append(os.path.join(eval_path,file_name))
    corpus=list()
    for file_name in file_path:
        corpus_single_file=list()
        judge_single_file=list()
        with open(file_name) as f:
            for line in f:
                if "show batch outputs" in line:
                    corpus_single_file.append(line[22:-4].strip().replace("\\'","'").replace("\\\\","\\").replace("\\\\","\\"))
                elif "completion" in line:
                    if line[13:-3] == "Yes":
                        judge_single_file.append(True)
                    else:
                        judge_single_file.append(False)
        jailbreak_item=[data for data, is_true in zip(corpus_single_file[-100:], judge_single_file[-100:]) if is_true]
        corpus.extend(jailbreak_item)

    corpus_file = model+folder+str(alpha)+'.txt'
    with open(corpus_file, "w") as file:
        for item in corpus:
            file.write(item + "\n")



    # 初始化 Self-BLEU 实例
    self_bleu = SelfBleu(test_text=corpus_file,gram=5)

    # 计算 Self-BLEU 分数（完整计算）
    score_full = self_bleu.get_score(is_fast=False)
    return score_full

In [16]:
res=get_self_bleu(model='vicuna',folder='_with_cosdecay_and_mean_sentemb',alpha=30)
print(f"Self-BLEU (Full): {res}")
res=get_self_bleu(model='vicuna',folder='_with_cosdecay_and_lasthiddenstate',alpha=10)
print(f"Self-BLEU (Full): {res}")
res=get_self_bleu(model='vicuna',folder='_without_cosdecay_and_mean_sentemb',alpha=15)
print(f"Self-BLEU (Full): {res}")
res=get_self_bleu(model='vicuna',folder='_without_cosdecay_and_lasthiddenstate',alpha=45)
print(f"Self-BLEU (Full): {res}")

Self-BLEU (Full): 0.5336163158498319
Self-BLEU (Full): 0.5397571228763103
Self-BLEU (Full): 0.5130230425715687
Self-BLEU (Full): 0.5164560569950525


In [17]:
import re
import os
model='llama2'
folders=['_with_cosdecay_and_lasthiddenstate','_without_cosdecay_and_lasthiddenstate','_with_cosdecay_and_mean_sentemb','_without_cosdecay_and_mean_sentemb']
alpha=0
corpus_path = "llama2_0_corpus.txt"
if os.path.exists(corpus_path):
    os.remove(corpus_path)

for folder in folders:
    rounds=os.listdir(model+folder)
    eval_paths=[ os.path.join(model+folder,round_,'eval_scripts','harmbench') for round_ in rounds]
    file_path=list()
    for eval_path in eval_paths:
        for file_name in os.listdir(eval_path):
            alpha_=int(re.findall(r'(?:search|alpha)_(\d*)_',file_name)[0])
            if alpha_==alpha:
                break
        file_path.append(os.path.join(eval_path,file_name))

    corpus=list()
    for file_name in file_path:
        corpus_single_file=list()
        judge_single_file=list()
        with open(file_name) as f:
            for line in f:
                if "show batch outputs" in line:
                    corpus_single_file.append(line[22:-4].strip().replace("\\'","'").replace("\\\\","\\").replace("\\\\","\\"))
                elif "completion" in line:
                    if line[13:-3] == "Yes":
                        judge_single_file.append(True)
                    else:
                        judge_single_file.append(False)
        jailbreak_item=[data for data, is_true in zip(corpus_single_file[-100:], judge_single_file[-100:]) if is_true]
        corpus.extend(jailbreak_item)

    with open(corpus_path, "a") as file:
        for item in corpus:
            file.write(item + "\n")

self_bleu = SelfBleu(test_text=corpus_path,gram=5)

# 计算 Self-BLEU 分数（完整计算）
score_full = self_bleu.get_score(is_fast=False)
print(f"Self-BLEU (Full): {score_full}")

Self-BLEU (Full): 0.7981972159229245


In [18]:
res=get_self_bleu(model='llama2',folder='_with_cosdecay_and_mean_sentemb',alpha=25)
print(f"Self-BLEU (Full): {res}")
res=get_self_bleu(model='llama2',folder='_with_cosdecay_and_lasthiddenstate',alpha=40)
print(f"Self-BLEU (Full): {res}")
res=get_self_bleu(model='llama2',folder='_without_cosdecay_and_mean_sentemb',alpha=5)
print(f"Self-BLEU (Full): {res}")
res=get_self_bleu(model='llama2',folder='_without_cosdecay_and_lasthiddenstate',alpha=40)
print(f"Self-BLEU (Full): {res}")

Self-BLEU (Full): 0.7274379223359446
Self-BLEU (Full): 0.7496960443687458
Self-BLEU (Full): 0.7564788189814915
Self-BLEU (Full): 0.7430622305937099
